In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [2]:
conf = SparkConf()
conf.set("spark.hadoop.fs.s3a.impl",
                 "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "123456789")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider',
            'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set(
    "spark.jars.packages",
    "io.delta:delta-core_2.12:1.0.0,"
    "org.apache.hadoop:hadoop-aws:3.1.1,"
    "com.amazonaws:aws-java-sdk:1.11.271,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.271,"
    "software.amazon.awssdk:url-connection-client:2.15.40",
)
conf.set("spark.sql.extensions",
            "io.delta.sql.DeltaSparkSessionExtension")
conf.set(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
conf.set("spark.databricks.delta.merge.repartitionBeforeWrite.enabled","true")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .master("local[*]")
    .getOrCreate()
)

23/03/21 09:05:49 WARN Utils: Your hostname, duc-G3-3579 resolves to a loopback address: 127.0.1.1; using 192.168.2.103 instead (on interface wlo1)
23/03/21 09:05:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/duc/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/duc/.ivy2/cache
The jars for the packages stored in: /home/duc/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4b7f2fb-c386-4305-a01a-6a44e706ddf9;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
	fo

In [24]:
orderDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.orders")
orderDF.orderBy("order_number").show()

+---+-------------+------------+----------+---------+--------+----------+-------------------+
| op|        ts_ms|order_number|order_date|purchaser|quantity|product_id|before_order_number|
+---+-------------+------------+----------+---------+--------+----------+-------------------+
|  r|1679370426459|       10001|     16816|     1001|       1|       102|              10001|
|  r|1679370426459|       10002|     16817|     1002|       2|       105|              10002|
|  r|1679370426460|       10003|     16850|     1002|       2|       106|              10003|
|  r|1679370426460|       10004|     16852|     1003|       1|       107|              10004|
|  u|1679371329793|       10005|     19037|     1003|    2000|       107|              10005|
+---+-------------+------------+----------+---------+--------+----------+-------------------+



In [22]:
productDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.products")
productDF.orderBy("id").show()

+---+-------------+---+------------------+--------------------+------+---------+
| op|        ts_ms| id|              name|         description|weight|before_id|
+---+-------------+---+------------------+--------------------+------+---------+
|  r|1679370426464|101|           scooter|Small 2-wheel sco...|  3.14|      101|
|  r|1679370426465|102|       car battery|     12V car battery|   8.1|      102|
|  r|1679370426465|103|12-pack drill bits|12-pack of drill ...|   0.8|      103|
|  r|1679370426465|104|            hammer|12oz carpenter's ...|  0.75|      104|
|  r|1679370426465|105|            hammer|14oz carpenter's ...| 0.875|      105|
|  r|1679370426465|106|            hammer|16oz carpenter's ...|   1.0|      106|
|  r|1679370426465|107|             rocks|box of assorted r...|   5.3|      107|
|  r|1679370426466|108|            jacket|water resistent b...|   0.1|      108|
|  r|1679370426466|109|        spare tire|  24 inch spare tire|  22.2|      109|
|  c|1679370647805|110|     

In [8]:
customerDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.customers")
customerDF.show()

+---+-------------+----+----------+---------+--------------------+
| op|        ts_ms|  id|first_name|last_name|               email|
+---+-------------+----+----------+---------+--------------------+
|  r|1679362979409|1001|     Sally|   Thomas|sally.thomas@acme...|
|  r|1679362979410|1004|      Anne|Kretchmar|  annek@noanswer.org|
|  r|1679362979409|1002|    George|   Bailey|  gbailey@foobar.com|
|  r|1679362979410|1003|    Edward|   Walker|       ed@walker.com|
+---+-------------+----+----------+---------+--------------------+



In [4]:
orderDF = spark.read.format("delta").load("s3a://datalake/sliver")
orderDF.show()

+--------------------+--------------------+-----------------+
|               email|           timestamp|     total_weight|
+--------------------+--------------------+-----------------+
|sally.thomas@acme...|2023-03-21 08:42:...|8.100000381469727|
|  gbailey@foobar.com|2023-03-21 08:42:...|            1.875|
|       ed@walker.com|2023-03-21 08:42:...|5.300000190734863|
+--------------------+--------------------+-----------------+



In [ ]:
path_table = "s3a://datalake/brozen/cdc.inventory.orders"
df = spark.read.format("delta").option("versionAsOf", 0).load(path_table)
df.show()

In [ ]:
delta_table = DeltaTable.forPath(spark, "s3a://datalake/brozen/cdc.inventory.orders")
history = delta_table.history()
history.show()

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("s3a://datalake/brozen/cdc.inventory.orders")
df.show()

In [ ]:
data = [(10004, 16852, 1003, 2000, 107, "u")]
columns = ["order_number", "order_date", "purchaser", "quantity", "product_id", "op"]
df = spark.createDataFrame(data, schema=columns)
df.show()

In [ ]:
orderDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.orders")
productDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.products")
customerDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.customers")

# orderDF.join(customerDF, customerDF.id == orderDF.purchaser, "inner") \
#                     .join(productDF, productDF.id == orderDF.product_id, "inner") \
#                     .selectExpr("order_number", "order_date as order_time", "email", "purchaser", 
#                                 "name as product_name", "quantity", "weight as unit_price") \
orderDF.writeStream.format("delta").option("checkpointLocation", "s3a://datalake/sliver/checkpoint/aggTable").outputMode("append").start("s3a://datalake/sliver/aggTable")

In [ ]:
df = spark.read.format("delta").load("s3a://datalake/sliver/aggTable")
df.show()

In [ ]:
delta_table = DeltaTable.forPath(spark, "s3a://datalake/sliver/aggTable")
history = delta_table.history()
history.show()

In [ ]:
df = spark.read.format("delta").load("s3a://datalake/sliver/calTable")
df.count()

In [32]:
orderDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.orders")
productDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.products")
customerDF = spark.readStream.option("ignoreChanges", "true").format("delta").load("s3a://datalake/brozen/cdc.inventory.customers")

joinDF = orderDF.join(customerDF, customerDF.id == orderDF.purchaser, "inner") \
                    .join(productDF, productDF.id == orderDF.product_id, "inner") \
                    .selectExpr("order_number", "order_date as order_time", "email", "purchaser", 
                                "name as product_name", "quantity", "weight as unit_price")

calDF = joinDF.withColumn(
  	    "total_price", joinDF.quantity * joinDF.unit_price)

total_spent_DF = calDF \
        .groupBy("email") \
        .agg(_sum("total_price")) 

# product_DF = calDF \
#             .groupBy("product_name") \
#             .agg(
#                 _sum("quantity").alias("products_selled"), 
#                 _sum("total_price").alias("total_price")
#             )

# product_price = calDF \
#             .groupBy("order_time","product_name") \
#             .agg(
#                 avg("unit_price").alias("ave_unit_price")
            # )

calDF.writeStream.format("delta").option("checkpointLocation", "s3a://datalake/sliver/checkpoint/calTable") \
                    .outputMode("append").start("s3a://datalake/sliver/calTable")

total_spent_DF.writeStream.format("delta").option("checkpointLocation", "s3a://datalake/sliver/checkpoint/total_spent_Df") \
                    .outputMode("append").start("s3a://datalake/sliver/total_spent_Df")

# product_DF.writeStream.format("delta").option("checkpointLocation", "s3a://datalake/sliver/checkpoint/product_DF") \
#                     .outputMode("append").start("s3a://datalake/sliver/product_DF")

# product_price.writeStream.format("delta").option("checkpointLocation", "s3a://datalake/sliver/checkpoint/product_price") \
#                     .outputMode("append").start("s3a://datalake/sliver/product_price")

23/03/17 15:10:00 WARN StreamingQueryManager: Stopping existing streaming query [id=c5f28b59-15cf-45b9-a5da-233245351362, runId=3c4bb777-cd3f-4b70-abd0-3d42989ac196], as a new run is being started.


AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [email#23682], [email#23682, sum(cast(total_price#23753 as double)) AS sum(total_price)#23771]
+- Project [order_number#23647, order_time#23743, email#23682, purchaser#23649, product_name#23744, quantity#23650, unit_price#23745, (cast(quantity#23650 as float) * unit_price#23745) AS total_price#23753]
   +- Project [order_number#23647, order_date#23648 AS order_time#23743, email#23682, purchaser#23649, name#23665 AS product_name#23744, quantity#23650, weight#23667 AS unit_price#23745]
      +- Join Inner, (id#23664 = product_id#23651)
         :- Join Inner, (id#23679 = purchaser#23649)
         :  :- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@6f692efb,delta,List(),None,List(),None,Map(ignoreChanges -> true, path -> s3a://datalake/brozen/cdc.inventory.orders),None), delta, [order_number#23647, order_date#23648, purcha

23/03/17 15:15:22 ERROR MicroBatchExecution: Query [id = 28dc1a50-aeaa-4cd8-9e82-9cd39dafa6bf, runId = 00879f97-05e2-4835-bd3b-93c1f141cc21] terminated with error
java.io.IOException: rename from s3a://datalake/sliver/checkpoint/calTable/offsets/.1.fe339b9c-6b90-4694-82b3-1290ff2ae67d.tmp to s3a://datalake/sliver/checkpoint/calTable/offsets/1 failed.
	at org.apache.hadoop.fs.FileSystem.rename(FileSystem.java:1548)
	at org.apache.hadoop.fs.DelegateToFileSystem.renameInternal(DelegateToFileSystem.java:204)
	at org.apache.hadoop.fs.AbstractFileSystem.renameInternal(AbstractFileSystem.java:769)
	at org.apache.hadoop.fs.AbstractFileSystem.rename(AbstractFileSystem.java:699)
	at org.apache.hadoop.fs.FileContext.rename(FileContext.java:1032)
	at org.apache.spark.sql.execution.streaming.FileContextBasedCheckpointFileManager.renameTempFile(CheckpointFileManager.scala:335)
	at org.apache.spark.sql.execution.streaming.CheckpointFileManager$RenameBasedFSDataOutputStream.close(CheckpointFileManager